# <b>CaRM Module: Advanced Topics in Data Preparation Using Python (2024/2025)</b>
## <b>Session 3: Cleaning and merging your data. </b>

### <b>Cleaning</b>

#### <b>3.1. Removing missing data</b>

You may remember that, when printing the information regarding mission1_data, several columns had a number of non-null values that was smaller than the total number of rows. This happens when the value is non available (missing data, never entered when collecting the data). The Non Available data is often referred as NA.

In [ ]:
import pandas as pd

df = pd.read_csv('mission1_data.csv')

print(df.info()) # Note that several columns have less than 87 non-null values.

If you inspect the data further, you may notice that some of the values are represented as NaN, which is the acronym for Non A Number. This happens not only in numeric columns but also in columns with text.

In [ ]:
import pandas as pd

df = pd.read_csv('mission1_data.csv')

print(df.sample(10))

To identify the specific values that are NANs and distinguish them from the existing values, use the .<b>isnull()</b> and <b>.notnull()</b> methods. These methods are useful to create a boolean series based on a specific column, which you can then use as a filter for your data. For example, let's say you would like to separate the data into creatures that have a defined death year on one hand, and those that haven't.

In [ ]:
import pandas as pd

df = pd.read_csv('mission1_data.csv')

# create a boolean series for creatures that do not have a defined death year (possibly still alive)
isalive = df.death_year.isnull()
print(isalive)

# create a boolean series for creatures that have a defined death year (possibly really dead)
isdead = df.death_year.notnull()
#print(isdead)

# Now, filter the data by selecting the rows of creatures that are presumably still alive
df_alive = df[isalive]
#print(df_alive[['death_year','death_era']])

# And, filter the data by selecting the rows of creatures that have died
df_dead = df[isdead]
#print(df_dead[['death_year','death_era']])


There is another method that allows to remove rows or columns containing NaNs in one step called <b>.dropna()</b>. This method has different arguments that enables some flexibility: for example, you may want to remove rows with NaNs, but only if there are NaNs in all columns or in a subset of columns. Let's say you would like to remove all the entries for creatures that do not have a defined birth year and death year.

In [ ]:
import pandas as pd

df = pd.read_csv('mission1_data.csv')

# If you attempt to remove the rows that contain NaNs in all the columns, 
# you end up with the same dataset in this case.
df_clean = df.dropna(how='all')
print(df_clean.shape)

# However, if you remove the rows that contain NaNs in at least one of the columns,
# you end up with very few rows.
df_clean = df.dropna(how='any')
#print(df_clean.shape)

# There could be columns that are mostly filled with NaNs, and are probably not relevant 
# for your analyses. See for example, the column cybernetics:
#print(df.info())
#print(df.cybernetics.isnull().value_counts())

# If you would like to remove those rows that contain NaNs in specific columns, use the 
# argument subset to specify a list with the columns of interest:
#df_clean = df.dropna(how='any',subset=['birth_year','death_year'])
#print(df_clean.shape)

# You could also apply the method to remove columns. For example, if you would like to 
# remove columns that contain more than 10 NaNs:
#df_clean = df.dropna(axis=1,thresh=10)
#print(df.shape)
#print(df.columns)
#print(df_clean.shape)
#print(df_clean.columns)
# by comparison of the original and clean datatets, you can see that cybernetics column 
# was removed using this criterion.

# check other arguments and possibilities with the help in-built function:
#help(df.dropna)

#### <b>3.2. Replacing missing data</b>

Sometimes you would like to replace the NA values with a specific term or number. There are two methods that allow this: <b>.fillna()</b>, and more generally <b>.replace()</b>. For example, you assume that those creatures without a concrete death year are still alive. You could replace the NaNs in this column with the term 'alive'.

In [ ]:
import pandas as pd

df = pd.read_csv('mission1_data.csv')

df.death_year.fillna('Alive', inplace=True)
print(df.death_year)

Note that we set the <b>inplace</b> is argument to True. The inplace argument is used by several functions in Pandas, but the default is False. Generally, you would not use the inplace argument, and the function will not modify the original dataframe, but return a modified copy. This is a safer procedure. However, sometimes you are sure that you would like to modify the original data (like in the example above), so you can call the inplace argument and set it to True.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('mission1_data.csv')

df.death_year = df.death_year.replace(to_replace=np.nan, value='Alive')
print(df.death_year)

NA values can also be represented as Null or empty strings. You can apply the methods .fillna() and .replace() also in these situations, with slight modifications.

In [ ]:
import pandas as pd

myDict = {
    'fruits': ['Apple', 'Orange', None, 'Melon'],
    'animals': ['Dog', '', 'Cow', 'Bird']
}

# Creating a DataFrame with null values
df = pd.DataFrame(myDict)
print('This is the original data with None and empty strings:')
print(df)

# Replace null values with a specified value
df.fillna('No Value', inplace=True) # Note that only None was replaced with 'No Value'
print('\nNow the None value was replaced with \'No Value\':')
print(df)

# Replace empty strings with a specified value
df.replace('', 'Empty', inplace=True)
print('\nNow the empty string was replaced with \'Empty\':')
print(df)

The <b>interpolate()</b> method is used to fill NaN values in the DataFrame or Series using various interpolation techniques. This is very useful for numeric data series, for example, because it allows to generate the missing data based on some mathematical algorithm.

In [ ]:
import pandas as pd
import numpy as np

# create a range of integers from 1 to 100
fake_numbers = np.arange(1,101)
#print(fake_numbers)

np.random.seed(0)
fake_data = np.random.randint(0,75,len(fake_numbers)).tolist()
fake_data2 = np.random.randint(-10,24,len(fake_numbers)).tolist()

# create the dataframe
df = pd.DataFrame({'measurement': fake_numbers,
                   'rain_mm': fake_data, 
                   'temperature': fake_data2})

# now, let's intentionally produce NaNs:
df.iloc[[4,8,50],1] = np.nan
df.iloc[[3,7,60],2] = np.nan
print(df.head(10))

df_new = df.interpolate(method ='linear')# #try pchip
#df_new = df.interpolate(method ='polynomial', order=2) #.round(1) # try spline 
print('\nInterpolated data:')
print(df_new.head(10))
#help(df.interpolate)


df.temperature.plot()
#df_new.temperature.plot()

#### <b>3.3. Detecting and removing duplicates</b>

You may remember that, in a previous session, we removed duplicates from a list using a simple trick. We converted first the list to a set data structure, which does not allow duplicates. Then, we converted the set back to a list object.

In [ ]:
my_list = [1, 2, 2, 3, 4, 4]
unique_items = list(set(my_list))
print(unique_items)

In Pandas DataFrames, you can detect duplicated rows with the <b>.duplicated()</b> method and remove duplicated rows with the <b>.drop_duplicate()</b> method. For example, in the example below you will find two repeated entries in the data, for Alice and Bob.

In [ ]:
import pandas as pd 
# Sample DataFrame with duplicate rows 
myDict = { 
    'Name': ['Alice', 'Bob', 'Charlie', 'Alice', 'Eva', 'Bob'], 
    'Age': [24, 27, 22, 24, 29, 27], 
    'Score': [85, 90, 78, 85, 92, 90] 
    } 

df = pd.DataFrame(myDict) 

# Display the original DataFrame 
print("Original DataFrame:") 
print(df) 

# Identifying duplicates using .duplicated() method 
duplicates = df.duplicated() 
print("\nDuplicated rows:") 
print(duplicates) 

# Dropping duplicates using .drop_duplicates() method 
df_no_duplicates = df.drop_duplicates() 
print("\nDataFrame after dropping duplicates:") 
print(df_no_duplicates)

You can also apply this method based on duplicated values found in a column. For example, imagine a situation when you would only like to keep an entry for each different value in a column of a dataframe.

In [ ]:
import pandas as pd

df = pd.read_csv('mission1_data.csv')

# sorting by species
#df.sort_values("species", inplace=True)

# This keeps only one creature of each species
df_one = df.drop_duplicates(subset="species")
print(df_one.shape)
print(df_one[['name','species']])

# This keeps only the creatures that are unique in their species
# Use the argument keep=False, to consider all of the same values as duplicates
df_unique = df.drop_duplicates(subset="species", keep=False) # keep can be first, last or False
print(df_unique.shape)
#print(df_unique[['name','species']])

### <b>Combining datasets</b>

#### <b>3.4. Concatenating data</b>

You may remember that we used Pandas <b>concat</b> method to combine two series and create a dataframe. This method can also be used to combine dataframes.

In [ ]:
import pandas as pd 
# Create two sample DataFrames 
df1 = pd.DataFrame({ 'numbers': [1,2,3,4], 'letters': ['a','b','c','d'] }) 
df2 = pd.DataFrame({ 'numbers': [5,6,7,8], 'letters': ['e', 'f', 'g', 'h'] }) 

# Display the original DataFrames 
print("DataFrame 1:") 
print(df1) 
print("\nDataFrame 2:") 
print(df2) 

# Concatenate DataFrames along rows (default behavior) 
result = pd.concat([df1, df2]) 
print("\nConcatenated DataFrame (along rows):") 
print(result) # Concatenate DataFrames along columns 

result_col = pd.concat([df1, df2], axis=1) 
print("\nConcatenated DataFrame (along columns):") 
print(result_col)

#### <b>3.5. Joining data</b>

It is also possible to concatenate dataframes with the <b>.join()</b> method of the dataframe itself. Importantly, in the simplest application, both dataframes should have the same indexes and no overlapping columns.

In [ ]:
import pandas as pd 
# Create two sample DataFrames 
df1 = pd.DataFrame({ 'numbers': [1,2,3,4], 'letters': ['a','b','c','d'] }, index=[0,1,2,3]) 
df2 = pd.DataFrame({ 'numbers_2': [5,6,7,8], 'letters_2': ['e', 'f', 'g', 'h'] }, index=[0,1,2,3]) 

# Display the original DataFrames 
print("DataFrame 1:") 
print(df1) 
print("\nDataFrame 2:") 
print(df2) 

# Join DataFrames
joined_df = df1.join(df2) 
print("\nJoined DataFrame:") 
print(joined_df)

If the dataframes have different indexes, the method will join the dataframes but will introduce NaNs in columns where there is missing data. This method also has the <b>how</b> argument, that allows to specify how to combine the data when the indexes (rows) are not completely overlapping.

In [ ]:
import pandas as pd 
# Create two sample DataFrames 
df1 = pd.DataFrame({ 'numbers': [1,2,3,4], 'letters': ['a','b','c','d'] }, index=[0,1,2,3]) 
df2 = pd.DataFrame({ 'numbers_2': [5,6,7,8], 'letters_2': ['e', 'f', 'g', 'h'] }, index=[0,1,3,4]) 

# Display the original DataFrames 
print("DataFrame 1:") 
#print(df1) 
print("\nDataFrame 2:") 
#print(df2) 

# Join DataFrames
joined_df = df1.join(df2) 
print("\nJoined DataFrame:") 
print(joined_df)

# Join DataFrames
joined_df = df1.join(df2, how='outer') 
print("\nJoined DataFrame for all rows:") 
print(joined_df)

# Join DataFrames
joined_df = df1.join(df2, how='inner') 
print("\nJoined DataFrame in overlapping rows only:") 
print(joined_df)

#### <b>3.6. Merging data</b>

<b>.merge()</b> is the Pandas method for combining data based on common columns or indices. It is more flexible than the <b>.join()</b> method of the dataframe. If both dataframes have the same indexes, the application of the method is quite straightforward.

In [ ]:
import pandas as pd 
# Create two sample DataFrames with non-overlapping columns
df1 = pd.DataFrame({ 'numbers': [1,2,3,4], 'letters': ['a','b','c','d'] }, index=[0,1,2,3]) 
df2 = pd.DataFrame({ 'numbers_2': [5,6,7,8], 'letters_2': ['e', 'f', 'g', 'h'] }, index=[0,1,2,3]) 

# Create two sample DataFrames with overlapping columns
#df1 = pd.DataFrame({ 'numbers': [1,2,3,4], 'letters': ['a','b','c','d'] }, index=[0,1,2,3]) 
#df2 = pd.DataFrame({ 'numbers': [5,6,7,8], 'letters': ['e', 'f', 'g', 'h'] }, index=[0,1,2,3]) 

# Display the original DataFrames 
print("DataFrame 1:") 
#print(df1) 
print("\nDataFrame 2:") 
#print(df2) 

# Join DataFrames
# if the columns don't overlap
merged_df = pd.merge(df1,df2, left_index=True, right_index=True)
# if the columns overlap
#merged_df = pd.merge(df1,df2, left_index=True, right_index=True, suffixes=('_2','_2')) 
print("\nMerged DataFrame:") 
print(merged_df)

If both dataframes have non overlapping indexes (rows) or values in a column, then it is necessary to use other arguments:<br>
<br>
<b>on</b> tells merge() which columns or indices, also called key columns or key indices, you want to join on. If you use on, then the column or index that you specify must be present in both objects.<br>
<br>
<b>how</b> defines what kind of merge to make. It defaults to 'inner', but other possible options include 'outer', 'left', and 'right', like we saw in the .join() method.<br>


In [ ]:
import pandas as pd 
# Create two sample DataFrames 
df1 = pd.DataFrame({ 'numbers': [1,2,3,4], 'letters': ['a','b','c','d'] }, index=[0,1,2,3]) 
df2 = pd.DataFrame({ 'numbers': [5,6,7,8], 'letters': ['e', 'f', 'g', 'h'] }, index=[4,5,6,7]) 
#df2 = pd.DataFrame({ 'numbers': [1,2,7,8], 'letters': ['e', 'f', 'g', 'h'] }, index=[4,5,6,7]) 

# Display the original DataFrames 
print("DataFrame 1:") 
print(df1) 
print("\nDataFrame 2:") 
print(df2) 

# Merge DataFrames
merged_df = pd.merge(df1,df2,on='numbers', how='outer') 
print("\nMerged DataFrame for all rows:") 
print(merged_df)

# Merge DataFrames
merged_df = pd.merge(df1,df2,on='numbers', how='inner')  
print("\nMerged DataFrame in overlapping rows only:") 
#print(merged_df)

# Merge DataFrames according to rows in left dataframe
merged_df = pd.merge(df1,df2,on='numbers', how='left')  
print("\nMerged DataFrame for rows found in left dataframe only:") 
#print(merged_df)

# Merge DataFrames according to rows in right dataframe
merged_df = pd.merge(df1,df2,on='numbers', how='right')  
print("\nMerged DataFrame for rows found in right dataframe only:") 
#print(merged_df)

### <b>Mission 3</b>

In [ ]:
import pandas as pd

df1 = pd.read_csv('mission3_data1.csv')
df2 = pd.read_csv('mission3_data2.csv')
df3 = pd.read_csv('mission3_data3.csv')

pd.set_option('display.max_rows',1000)
print(df1.info())
print(df2.info())
print(df3.info())

<b>Excercises with mission data</b>

1. Combine the Planet and Sector columns of datasets 2 and 3 into a single column called 'Planet or sector represented', including Planet, Sector information.

2. Rename column 'Planet or sector represented' in all datasets as 'Place'.

3. Split the 'Species and gender' column into two columns: 'Species' and 'Gender'.

4. Combine the three datasets to contain all the data.

5. Replace all NA values in column Notes as 'Nothing to report'.

In [ ]:
import pandas as pd
import numpy as np
import re

df1 = pd.read_csv('mission3_data1.csv')
df2 = pd.read_csv('mission3_data2.csv')
df3 = pd.read_csv('mission3_data3.csv')

def planetsector(x):
    if isinstance(x['Sector'], str):
        y = str(x['Planet']) + ', ' + x['Sector']
    else:
        y = x['Planet']
    return y

df3.replace('--', np.nan, inplace=True)

df2['Planet or sector represented'] = df2.apply(planetsector, axis=1)
df3['Planet or sector represented'] = df3.apply(planetsector, axis=1)

df2.drop(['Planet', 'Sector'], axis=1, inplace=True)
df3.drop(['Planet', 'Sector'], axis=1, inplace=True)

df1.rename(columns={'Planet or sector represented': 'Place'}, inplace=True)
df2.rename(columns={'Planet or sector represented': 'Place'}, inplace=True)
df3.rename(columns={'Planet or sector represented': 'Place'}, inplace=True)

def getspecies(x):
    if isinstance(x, str):
        newx = re.sub("female","", x)
        newx = re.sub("Female","", newx)
        newx = re.sub("male","", newx)
        newx = re.sub("Male","", newx)
        return newx
    else:
        return np.nan

df1['Species'] = df1['Species and gender'].apply(getspecies)
df2['Species'] = df2['Species and gender'].apply(getspecies)
df3['Species'] = df3['Species and gender'].apply(getspecies)

def getgender(x):
    if isinstance(x, str):
        if re.search('female', x.lower()):
            return 'Female'
        elif re.search('male', x.lower()):
            return 'Male'
        else:
            return np.nan
        
df1['Gender'] = df1['Species and gender'].apply(getgender)
df2['Gender'] = df2['Species and gender'].apply(getgender)
df3['Gender'] = df3['Species and gender'].apply(getgender)

df1.drop(['Species and gender'], axis=1, inplace=True)
df2.drop(['Species and gender'], axis=1, inplace=True)
df3.drop(['Species and gender'], axis=1, inplace=True)

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',100)
pd.set_option('display.width',1000)
#print(df1)
#print(df3)

dfcomb = pd.concat([df1,df2,df3]) 
#print(dfcomb.shape)
#print(dfcomb)

# of course, you could have performed some operations above after combining the dataframes 
dfcomb['Notes'] = dfcomb.Notes.apply(lambda x: x if isinstance(x, str) else 'Nothing to report')
print(dfcomb)

     Unnamed: 0                             Senator                                              Place                                               Term                                              Notes                        Species  Gender
0             0                                Aang                                              Roona                                          c. 21 BBY                                  Nothing to report                        Roonan     Male
1             1                            Aak, Ask                    Malastare and the Dustig sector                                      22 BBY–19 BBY                                  Nothing to report                          Gran     Male
2             2                 Adem'thorn, Yeb Yeb                   Makem Te and the Nilgaard sector                                          c. 32 BBY                                  Nothing to report                 Swokes Swokes     Male
3             3         